In [1]:
import numpy as np
import pandas as pd
import time
import tqdm
from sklearn import preprocessing

df = pd.read_csv('train_data.csv')
df = df.fillna('')
X_diag = df['Диагноз']
X_ana = df['Анамнез заболевания']
X_vnosm = df['Внешний осмотр']
X_age = df['Возраст'].values
X_sex = df['Пол'].values

#df_pr_1 = pd.read_csv('train_pred_1.csv', sep=',')
#df_pr_2 = pd.read_csv('train_pred_2.csv', sep=',')
#X_pr_1 = df_pr_1['proba'].values
#X_pr_2 = df_pr_2['proba'].values

y = df['Revisit']
df.head()

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit
0,0,J06.0,Острый ларингофарингит,29,2,,,,На жевательной поверхности 2.6- кариозная поло...,1
1,1,N76.1,Подострый и хронический вагинит,45,2,,,считает себя больной на протяжении многих лет....,,1
2,2,L23.9,"Аллергический контактный дерматит, причина не ...",63,2,,,"Болен около 2-х дней, когда появились описанны...",,1
3,3,N48.1,Баланопостит,43,2,,не отягощен,,,1
4,4,Z00.0,Общий медицинский осмотр,29,1,,,провела полное обследование для оперативного л...,,1


In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
n_samples = len(X_diag.values)
print(n_samples)
corpus = np.hstack((X_diag.values, X_vnosm))
vectorizer = CountVectorizer(min_df = 3/(2*len(corpus)), token_pattern = '[0-9]*[а-яА-ЯёЁa-zA-Z]+',
                             max_features = 10000, ngram_range=(1,2))
#vectorizer = CountVectorizer(min_df = 3/(2*len(corpus)), max_features = 10000, stop_words = ['NaN'])
print(corpus.shape)
D = vectorizer.fit_transform(corpus)
dict = vectorizer.get_feature_names()
print(len(dict))

283086
(566172,)
9954


In [3]:
print('df', len(df))

ana = X_ana.dropna(axis = 0)
print('alerg', len(df['аллергия'].dropna(axis = 0)))
print('osmotr', len(df['Внешний осмотр'].dropna(axis = 0)))
print('Sost', len(df['Общее состояние'].dropna(axis = 0)))
print('Anamn', len(ana))

3/(2*len(corpus))

df 283086
alerg 283086
osmotr 283086
Sost 283086
Anamn 283086


2.649371569063818e-06

In [4]:
df.describe()

,ID,Возраст,Пол,Revisit
count,283086.000000,283086.000000,283086.000000,283086.000000
mean,141542.500000,41.696905,1.708347,0.828639
std,81720.033489,21.540704,0.454524,0.376825
min,0.000000,1.000000,1.000000,0.000000
25%,70771.250000,31.000000,1.000000,1.000000
50%,141542.500000,38.000000,2.000000,1.000000
75%,212313.750000,51.000000,2.000000,1.000000
max,283085.000000,2017.000000,2.000000,1.000000


In [5]:
df.select_dtypes('object')['Диагноз'].value_counts()

Вагинит, вульвит и вульвовагинит при инфекционных и паразитарных болезнях, классифицированных в других рубриках    11214
Острый вагинит                                                                                                     10119
Остеохондроз позвоночника у взрослых                                                                                8661
Острая инфекция верхних дыхательных путей неуточненная                                                              6119
Хронический простатит                                                                                               4373
Подострый и хронический вагинит                                                                                     4218
Цервикалгия                                                                                                         3730
Острый назофарингит (насморк)                                                                                       3699
Эрозия и эктропион шейки матки  

In [6]:
[t for t in df if type(t)]

['ID',
 'Код_диагноза',
 'Диагноз',
 'Возраст',
 'Пол',
 'Общее состояние',
 'аллергия',
 'Анамнез заболевания',
 'Внешний осмотр',
 'Revisit']

In [7]:
df['Анамнез заболевания'].apply(lambda x: np.log(len(x)+1)).hist(bins=100)

In [8]:
pd.options.display.max_colwidth = 1000

In [9]:
df[['Анамнез заболевания']].replace('NaN', np.nan).fillna('')

,Анамнез заболевания
0,
1,считает себя больной на протяжении многих лет. Появилась боль и скованность в спине. По словам паицентки в настоящее время занимается конным спортом и появляется боль и скованность в поясничном отделе позвончника. В ближайшее время пациентка планирует беременность и поэтому обратилась к неврологу.
2,"Болен около 2-х дней, когда появились описанные жалобы. Самостоятельно не лечился. обратился в МХЦ."
3,
4,провела полное обследование для оперативного лечения
5,
6,"проводит лечение, прибыла для коррекции назначений."
7,"Вышеописанные жалобы беспокоят с 03.12.17. Обращался к неврологу, мануальному терапевту. Прошел 5 сеансов мануальной терапии. Афобазол принимает ситуативно, настроение улучшилось, тревога меньше."
8,
9,Ранее к эндокринологу не обращалась и не обследовалась.


In [10]:
df.loc[1, 'Анамнез заболевания']

'считает себя больной на протяжении многих лет.  Появилась боль и скованность в спине.  По словам паицентки в настоящее время занимается конным спортом и появляется боль и скованность в поясничном отделе позвончника.  В ближайшее время пациентка планирует беременность и поэтому обратилась  к неврологу.'

In [6]:
import pymorphy2
m = pymorphy2
#from pymorphy import get_morph

In [7]:
morph = pymorphy2.MorphAnalyzer()
#OpencorporaTag.cyr_repr

In [15]:
# пол: мужской - masc, женский - femn, средний - neut
# время: прошедшее - past, настоящее - pres, будущее - futr
# часть речи: прилагательное - ADJF, сущ - NOUN, глагол - VERB, местоимение -  NPRO,предлог - PREP, союз - CONJ
# 



w = morph.parse('скованность')[1]

print(w.tag.cyr_repr)
print(w.normal_form)
print(w.tag.gender)
print(w.tag.tense)
print(w.tag.POS)

СУЩ,неод,жр ед,вн
скованность
femn
None
NOUN


In [14]:
words = df.loc[1, 'Анамнез заболевания'].split()
words

['считает',
 'себя',
 'больной',
 'на',
 'протяжении',
 'многих',
 'лет.',
 'Появилась',
 'боль',
 'и',
 'скованность',
 'в',
 'спине.',
 'По',
 'словам',
 'паицентки',
 'в',
 'настоящее',
 'время',
 'занимается',
 'конным',
 'спортом',
 'и',
 'появляется',
 'боль',
 'и',
 'скованность',
 'в',
 'поясничном',
 'отделе',
 'позвончника.',
 'В',
 'ближайшее',
 'время',
 'пациентка',
 'планирует',
 'беременность',
 'и',
 'поэтому',
 'обратилась',
 'к',
 'неврологу.']

In [38]:
#lst = s.split() - разбиение на слова по () тому что в скобках, т.е. тут по пробелу
words = df.loc[1, 'Анамнез заболевания'].split()
for word in words:
    w = morph.parse(word)[0]
    print(w.tag)
    print(w.normal_form)
    print(w.tag.gender)
    print(w.tag.tense)
    print(w.tag.POS) 
    

VERB,impf,tran sing,3per,pres,indc
считать
None
pres
VERB
NPRO,Anph sing,accs
себя
None
None
NPRO
ADJF,Subx,Qual femn,sing,datv
больной
femn
None
ADJF
PREP
на
None
None
PREP
NOUN,inan,neut sing,loct
протяжение
neut
None
NOUN
ADJF,Apro plur,gent
многий
None
None
ADJF
UNKN
лет.
None
None
None
VERB,perf,intr femn,sing,past,indc
появиться
femn
past
VERB
NOUN,inan,femn sing,accs
боль
femn
None
NOUN
CONJ
и
None
None
CONJ
NOUN,inan,femn sing,nomn
скованность
femn
None
NOUN
PREP
в
None
None
PREP
UNKN
спине.
None
None
None
PREP
по
None
None
PREP
NOUN,inan,neut plur,datv
слово
neut
None
NOUN
NOUN,anim,femn sing,gent
паицентка
femn
None
NOUN
PREP
в
None
None
PREP
NOUN,inan,neut sing,accs
настоящее
neut
None
NOUN
NOUN,inan,neut sing,accs
время
neut
None
NOUN
VERB,impf,intr sing,3per,pres,indc
заниматься
None
pres
VERB
ADJF,Qual masc,sing,ablt
конный
masc
None
ADJF
NOUN,inan,masc sing,ablt
спорт
masc
None
NOUN
CONJ
и
None
None
CONJ
VERB,impf,intr sing,3per,pres,indc
появляться
None
pres
VERB
NOUN,i

In [119]:
#Anamnez_words = df['Анамнез заболевания'].split()

In [163]:
gender = [[] for i in range(10)]
print(gender)
tense = [[] for i in range(10)]
for j in range(10):
    words = df.loc[j, 'Анамнез заболевания'].split()
    print(words)
    for word in words:
        print(word)
        if len(words) != 0:
            w = morph.parse(word)[0]
            if w.tag.POS == 'VERB':
                if w.tag.gender != None:
                    gender[j].append(w.tag.gender)
                    print (gender[j], j)
                if w.tag.tense != None:
                    tense[j].append(w.tag.tense)
            
        
    """print(w.tag.cyr_repr)
    print(w.normal_form)
    print(w.tag.gender)
    print(w.tag.tense)"""

[[], [], [], [], [], [], [], [], [], []]
[]
['считает', 'себя', 'больной', 'на', 'протяжении', 'многих', 'лет.', 'Появилась', 'боль', 'и', 'скованность', 'в', 'спине.', 'По', 'словам', 'паицентки', 'в', 'настоящее', 'время', 'занимается', 'конным', 'спортом', 'и', 'появляется', 'боль', 'и', 'скованность', 'в', 'поясничном', 'отделе', 'позвончника.', 'В', 'ближайшее', 'время', 'пациентка', 'планирует', 'беременность', 'и', 'поэтому', 'обратилась', 'к', 'неврологу.']
['femn'] 1
['femn', 'femn'] 1
['Болен', 'около', '2-х', 'дней,', 'когда', 'появились', 'описанные', 'жалобы.', 'Самостоятельно', 'не', 'лечился.', 'обратился', 'в', 'МХЦ.']
['masc'] 2
[]
['провела', 'полное', 'обследование', 'для', 'оперативного', 'лечения']
['femn'] 4
[]
['проводит', 'лечение,', 'прибыла', 'для', 'коррекции', 'назначений.']
['femn'] 6
['Вышеописанные', 'жалобы', 'беспокоят', 'с', '03.12.17.', 'Обращался', 'к', 'неврологу,', 'мануальному', 'терапевту.', 'Прошел', '5', 'сеансов', 'мануальной', 'терапии.', 'Аф

In [159]:
gender

[[],
 ['femn', 'femn'],
 ['masc'],
 [],
 ['femn'],
 [],
 ['femn'],
 ['masc', 'masc'],
 [],
 ['femn']]

In [161]:
tense

[[],
 ['pres', 'past', 'pres', 'pres', 'pres', 'past'],
 ['past', 'past'],
 [],
 ['past'],
 [],
 ['pres', 'past'],
 ['pres', 'past', 'past', 'pres'],
 [],
 ['past']]

In [97]:
#info = morph.get_graminfo(df.loc[1, 'Анамнез заболевания'])

In [166]:
df.loc[7, 'Анамнез заболевания']

'Вышеописанные жалобы беспокоят с 03.12.17.  Обращался к неврологу, мануальному терапевту. Прошел 5 сеансов мануальной терапии. Афобазол принимает ситуативно, настроение улучшилось, тревога меньше.'

In [35]:
newcol = pd.DataFrame(columns=['women', 'men', 'past', 'present', 'future'])
df.append(newcol, ignore_index = True).head(2)

,ID,Revisit,future,men,past,present,text,women,Анамнез заболевания,Внешний осмотр,Возраст,Диагноз,Код_диагноза,Общее состояние,Пол,аллергия
0,0.0,1.0,NaN,NaN,NaN,NaN,,NaN,,"На жевательной поверхности 2.6- кариозная полость средней глубины, выполненная пигментированным дентином. Зондирование слегка болезненно по эмалево-дентинной границе. Реакция на температурные раздражители отсутствует.Перкуссия б/б.",29.0,Острый ларингофарингит,J06.0,,2.0,
1,1.0,1.0,NaN,NaN,NaN,NaN,считать себя больной на протяжение многий год появиться боль и скованность в спина по слово паицентка в настоящее время заниматься конный спорт и появляться боль и скованность в поясничный отдел позвончник в ближний время пациентка планировать беременность и поэтому обратиться к невролог,NaN,считает себя больной на протяжении многих лет. Появилась боль и скованность в спине. По словам паицентки в настоящее время занимается конным спортом и появляется боль и скованность в поясничном отделе позвончника. В ближайшее время пациентка планирует беременность и поэтому обратилась к неврологу.,,45.0,Подострый и хронический вагинит,N76.1,,2.0,


In [8]:
t1 = time.time()
def text(text):
    # к нижнему регистру
    text = text.lower()
    
    # оставляем в предложении только русские буквы (таким образом
    # удалим и ссылки, и имена пользователей, и пунктуацию и т.д.)
    alph = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
    
    cleaned_text = ''
    for char in text:
        if (char.isalpha() and char[0] in alph) or (char == ' '):
            cleaned_text += char
        
    result = []
    result.append(cleaned_text)
                              
    return ' '.join(result)

df['text'] = df['Анамнез заболевания'].apply(text)

t2 = time.time()
print((t2 - t1)/60)

0.2482694347699483


In [9]:
df.head(5)

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit,text
0,0,J06.0,Острый ларингофарингит,29,2,,,,На жевательной поверхности 2.6- кариозная поло...,1,
1,1,N76.1,Подострый и хронический вагинит,45,2,,,считает себя больной на протяжении многих лет....,,1,считает себя больной на протяжении многих лет ...
2,2,L23.9,"Аллергический контактный дерматит, причина не ...",63,2,,,"Болен около 2-х дней, когда появились описанны...",,1,болен около х дней когда появились описанные ж...
3,3,N48.1,Баланопостит,43,2,,не отягощен,,,1,
4,4,Z00.0,Общий медицинский осмотр,29,1,,,провела полное обследование для оперативного л...,,1,провела полное обследование для оперативного л...


In [43]:
"""t1 = time.time()
def func(text):
    res = 0
        
    result = []
    for w in text.split():
        word = morph.parse(w)[0]
        if 'VERB' in word.tag:
            if 'past' in word.tag:
                past += 1
"""         """   if 'pres' in word.tag:
                pres += 1
            if 'futr' in word.tag:
                fut += 1
            if 'masc' in word.tag:
                sex_m += 1
            if 'femn' in word.tag:
                sex_f += 1""""""
        result.append(res)
    return ' '.join(result)
#result.append(morph.parse(word)[0].normal_form)
                              
    

df['past'] = df['Анамнез заболевания'].apply(func)

t2 = time.time()
print(t2 - t1)"""

IndentationError: unexpected indent (<ipython-input-43-03dcfa04808d>, line 19)

In [10]:
t1 = time.time()
def func_past(text):
    past = 0
        
    result = []
    for w in text.split():
        word = morph.parse(w)[0]
        if 'VERB' in word.tag:
            if 'past' in word.tag:
                past += 1
        #result.append(past)
    return past
#result.append(morph.parse(word)[0].normal_form)
                              
    

df['past'] = df['text'].apply(func_past)

t2 = time.time()
print((t2 - t1)/60)

25.828490841388703


In [11]:
df.head(2)

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit,text,past
0,0,J06.0,Острый ларингофарингит,29,2,,,,На жевательной поверхности 2.6- кариозная поло...,1,,0
1,1,N76.1,Подострый и хронический вагинит,45,2,,,считает себя больной на протяжении многих лет....,,1,считает себя больной на протяжении многих лет ...,2


In [12]:
t1 = time.time()
def func_pres(text):
    pres = 0
        
    result = []
    for w in text.split():
        word = morph.parse(w)[0]
        if 'VERB' in word.tag:
            if 'pres' in word.tag:
                pres += 1
        #result.append(pres)
    return pres
#result.append(morph.parse(word)[0].normal_form)
                              
    

df['pres'] = df['text'].apply(func_pres)

t2 = time.time()
print((t2 - t1)/60)

23.429331390062966


In [13]:
df.head(2)

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit,text,past,pres
0,0,J06.0,Острый ларингофарингит,29,2,,,,На жевательной поверхности 2.6- кариозная поло...,1,,0,0
1,1,N76.1,Подострый и хронический вагинит,45,2,,,считает себя больной на протяжении многих лет....,,1,считает себя больной на протяжении многих лет ...,2,4


In [14]:
t1 = time.time()
def func_futr(text):
    futr = 0
        
    #result = []
    for w in text.split():
        word = morph.parse(w)[0]
        if 'VERB' in word.tag:
            if 'futr' in word.tag:
                futr += 1
    return futr
#result.append(morph.parse(word)[0].normal_form)
                              
    

df['futr'] = df['text'].apply(func_futr)

t2 = time.time()
print((t2 - t1)/60)

24.33519412279129


In [15]:
df.head(2)

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit,text,past,pres,futr
0,0,J06.0,Острый ларингофарингит,29,2,,,,На жевательной поверхности 2.6- кариозная поло...,1,,0,0,0
1,1,N76.1,Подострый и хронический вагинит,45,2,,,считает себя больной на протяжении многих лет....,,1,считает себя больной на протяжении многих лет ...,2,4,0


In [16]:
t1 = time.time()
def func_male(text):
    male = 0
        
    for w in text.split():
        word = morph.parse(w)[0]
        if 'VERB' in word.tag:
            if 'masc' in word.tag:
                male += 1
    return male
#result.append(morph.parse(word)[0].normal_form)
                              
    

df['male'] = df['text'].apply(func_male)

t2 = time.time()
print((t2 - t1)/60)

29.49491150776545


In [17]:
df.head(2)

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit,text,past,pres,futr,male
0,0,J06.0,Острый ларингофарингит,29,2,,,,На жевательной поверхности 2.6- кариозная поло...,1,,0,0,0,0
1,1,N76.1,Подострый и хронический вагинит,45,2,,,считает себя больной на протяжении многих лет....,,1,считает себя больной на протяжении многих лет ...,2,4,0,0


In [18]:
t1 = time.time()
def func_female(text):
    female = 0
        
    for w in text.split():
        word = morph.parse(w)[0]
        if 'VERB' in word.tag:
            if 'femn' in word.tag:
                female += 1
    return female
#result.append(morph.parse(word)[0].normal_form)
                              
    

df['female'] = df['text'].apply(func_female)

t2 = time.time()
print((t2 - t1)/60)

22.711022901535035


In [19]:
df.head()

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit,text,past,pres,futr,male,female
0,0,J06.0,Острый ларингофарингит,29,2,,,,На жевательной поверхности 2.6- кариозная поло...,1,,0,0,0,0,0
1,1,N76.1,Подострый и хронический вагинит,45,2,,,считает себя больной на протяжении многих лет....,,1,считает себя больной на протяжении многих лет ...,2,4,0,0,2
2,2,L23.9,"Аллергический контактный дерматит, причина не ...",63,2,,,"Болен около 2-х дней, когда появились описанны...",,1,болен около х дней когда появились описанные ж...,3,0,0,2,0
3,3,N48.1,Баланопостит,43,2,,не отягощен,,,1,,0,0,0,0,0
4,4,Z00.0,Общий медицинский осмотр,29,1,,,провела полное обследование для оперативного л...,,1,провела полное обследование для оперативного л...,1,0,0,0,1


In [20]:
df.to_csv('gender-tense_data.csv', index = False)

In [22]:
t1 = time.time()
def lemm(text):

        
    result = []
    for word in cleaned_text.split():
        # лемматизируем
        result.append(morph.parse(word)[0].normal_form)
                              
    return ' '.join(result)

df['lemm'] = df['text'].apply(lemm)

t2 = time.time()
print((t2 - t1)/60)

NameError: name 'cleaned_text' is not defined

In [28]:
import warnings
warnings.filterwarnings("ignore")
import artm
import nltk
import feedparser
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vkady_000\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [7]:
morph = pymorphy2.MorphAnalyzer()
#OpencorporaTag.cyr_repr

In [8]:
data = pd.read_csv('gender-tense_data.csv');

In [9]:
data.tail()

,Unnamed: 0,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit,lemm,text,past,pres,futr,male,female
283081,283081,283081,E04.1,Нетоксический одноузловой зоб,46,2,NaN,NaN,Травма 3нед. назад - упала на улице. Обращалас...,NaN,1,травма нед назад упасть на улица обращаться в ...,травма нед назад упала на улице обращалась в ...,2,0,0,0,2
283082,283082,283082,Z01.4,Гинекологическое обследование (общее) (рутинное),68,2,Состояние удовлетворительное .,не отягощен,NaN,NaN,1,NaN,NaN,0,0,0,0,0
283083,283083,283083,H16,Кератит,36,1,NaN,Аллергию на лекарства - отрицает,NaN,NaN,1,NaN,NaN,0,0,0,0,0
283084,283084,283084,J00,Острый назофарингит (насморк),33,2,NaN,NaN,обратилась для осмотра при беременности 35 нед...,NaN,1,обратиться для осмотр при беременность неделя ...,обратилась для осмотра при беременности недел...,3,0,0,0,3
283085,283085,283085,N11,Хронический тубулоинтерстициальный нефрит,42,2,NaN,NaN,без дополнений.,NaN,1,без дополнение,без дополнений,0,0,0,0,0


In [10]:
corpus = np.hstack((data['lemm'].values))

In [30]:
stop_eng = nltk.corpus.stopwords.words('english')
stop_rus = nltk.corpus.stopwords.words('russian')

In [11]:
print(len(corpus))

283086


In [12]:
data.select_dtypes('object')['lemm'].value_counts()

консультация по провести обследование и коррекция лечение                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [13]:
corpus.dict_size()

AttributeError: 'numpy.ndarray' object has no attribute 'dict_size'

In [32]:
dic = set(corpus)

{'заболеть остро около день назад орвать болезненность в горло головной боль температура самостоятельно лечиться плоскание полость рот мирамистина динамик слабоположительный обратиться к лорврач для определение дальнейший тактика ведение',
 'на протяжение последний неделя отмечать усиление бель гнойничковый высыпание на лицо усиливаться перед месячный',
 'дать жалоба беспокоить длительно с детство обострение около раз в год последний около месяц назад обратиться к лорврач для определение дальнейший тактика ведение',
 'у пациентка острый цистит закончить курс лечение супраксом в стадия лечение уроваксомый провести обследование',
 'образование в течение неск год около день назад травмировать образование на кожа правый предплечье на момент осмотр беспокоить болезненность при пальпация',
 'в течение последний месяц вновь беспокоить сметанообразный выделение из половый путь',
 'болеть с год травма инфекция отрицать появление боль ни с чем не связывать выполнять мрт оба тбедренный сустав от 

143346

In [63]:
df[df['Анамнез заболевания'].str.match('.*[Мм]ениск')]

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit
52,52,Z00.8,Другие общие осмотры,29,2,NaN,NaN,"Со слов боли в коленном суставе беспокоят в течении недели. МРТ - Заключение: МР-картина образования дистального метадиафиза бедренной кости. Может соответствовать гигантоклеточной опухоли ?( остеобластома, остеокластома). Требует уточнения. Сопоставить с КТ исследованием нижней трети левой бедренной кости. Возможно дообследование МРТ с контрастным усилением. Дистрофические, статические, постконтузионные изменения в заднем роге медиального мениска ( I ст. по Stoller). Постконтузионные изменения передней крестообразной связки. Тендинит сухожилия квадрицепса. Лигаментит собственной связки надколенника. Рекомендована консультация ортопеда-травматолога",NaN,1
154,154,M13.9,Артрит неуточненный,54,2,NaN,NaN,"Со слов боли беспокоят в течении 20 дней. Боль носит периодический характер, усиливается при физической нагрузке. МРТ ЗАКЛЮЧЕНИЕ. МР-признаки надрыва передне-медиальной части переднего рога медиального мениска. Конгломерат параменискальных медиальных передних синовиальных кист. Воспаление жирового тела Гоффа. Теновагинит подколенной мышцы. Хондромаляция медиального мыщелка бедренной кости. Гипертрофия медипателлярной складки. Синовиит, супрапателлярный бурсит, капсулит.",NaN,0
228,228,N30.0,Острый цистит,35,2,NaN,NaN,"Боли беспокоят в течении нескольких месяцев, настоящее обострение в течении недели ЗАКЛЮЧЕНИЕ: МР-признаки повреждения немногочисленных волокон сухожилия четырехглавой мышцы бедра и латерального поддерживателя надколенника / хронический период /, тендиноза собственной связки надколенника; дегенеративных изменений менисков (II ст изменения МРС по Stoller). МР-признаки гонартроза /II ст/ правого коленного сустава; умеренного экссудативного синовита.",NaN,1
325,325,I83.9,Варикозное расширение вен нижних конечностей без язвы или воспаления,54,2,NaN,NaN,"Травма в быту 16.03.15. Упал со сноуборда. Обращался за помощью. Проводили аспирацию гемартроза левого коленного сустава. Получал кеторол. иммобилизацию ортезом. Выполнено МРТ левого коленного сустава 30.03.15: МР-картина объёмной подкожной гематомы передне-наружной поверхности сустава по ходу латерального удерживателя надколенника, отека подкожно-жировой клетчатки всей наружной поверхности левого коленного сустава, небольшого суставного выпота. МР-данных за повреждение менисков, связок коленного сустава не получено. УЗИ левого коленного сустава от 25.03.15: УЗИ признаки повреждения боковых связок с выпотом. Гематома мягких тканей голени.",NaN,1
1124,1124,Z01.8,Другое уточненное специальное обследование,41,2,NaN,NaN,"Болеет с 2015 года. В процессе тренировки по борьбе получил травму. Выполнял МРТ левого коленного сустава от 6.02.16: МР - картина неполного разрыва медиального мениска в области тела и заднего рога 3Аст, артроза коленного сустава 1-2 ст., умеренный суставной выпот. Лечение не получает.",NaN,1
1194,1194,Z01.4,Гинекологическое обследование (общее) (рутинное),36,2,NaN,NaN,"Со слов травма КС в 1998, подвернула голень в быту. Лечилась оперативно - АСК левого коленного сустава в 2004 (мениск\медиапателлярная складка ?) (выписки нет). С 2009 стала отмечать боль в КС после нагрузки. Лечилась НПВС с временным положительным результатом. Отмечала ""вывихи"" надколенника, устраняла самостоятельно. Боль в левой стопе с ноября 2016, стала отмечать боль без видимой причины, хотя длительное время отмечает деформацию стоп. Лечилась НПВС, стельки с умеренно положительной динамикой, болевой синдром сохраняется. 08.01.17 МР-картина комплекса анатомических особенностей строения пателло-феморального сустава, которые способствуют пателло-феморальной нестабильности. Хондромаляция надколенника II ст. Отек подушки Гоффа, возможно, в результате ее импиджмента на фоне нестабильности надколенника. Умеренно выраженные дегенеративные изменения заднего рога медиального мениска и минимально выраженные дегенеративные изменения обоих рогов л

In [22]:
#print(dict)

In [23]:
vocab_size = len(dict)

In [24]:
from keras import Model
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
import keras

import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
print(X_pr_1)

[0.96859711 0.87395597 0.77679843 ... 0.95728379 0.864806   0.86535209]


In [26]:
from sklearn.model_selection import train_test_split
import scipy
#X_sex = np.asarray(preprocessing.scale(X_sex))
X_sex = X_sex[..., np.newaxis]
X_age = np.asarray(preprocessing.scale(X_age))
X_age = X_age[..., np.newaxis]
X_pr_1 = X_pr_1[..., np.newaxis]
X_pr_2 = X_pr_2[..., np.newaxis]
D_1 = D[:n_samples]
D_2 = D[n_samples:2*n_samples]
D_3 = D[-n_samples:]
DATA = scipy.sparse.hstack((D_1, D_2, X_sex, X_age, X_pr_1, X_pr_2))
print(DATA.shape)
#X_train, X_test, y_train, y_test = train_test_split(DATA, y, test_size=0.005)
y = keras.utils.to_categorical(y)
#y_train = keras.utils.to_categorical(y_train)
#y_test = keras.utils.to_categorical(y_test)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


(283086, 19914)


In [29]:
model = Sequential()
model.add(Dense(500, input_shape=(DATA.shape[1],)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.7))

#model.add(Dense(700))
#model.add(BatchNormalization())
#model.add(Activation('relu'))
#model.add(Dropout(0.5))

#model.add(Dense(50))
#model.add(BatchNormalization())
#model.add(Activation('relu'))
#model.add(Dropout(0.3))

model.add(Dense(2))
model.add(Activation('softmax'))

In [ ]:
#model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(DATA.tocsr(), y, validation_split=0.3, epochs=2, batch_size=1000)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 500)               9957500   
_________________________________________________________________
batch_normalization_3 (Batch (None, 500)               2000      
_________________________________________________________________
activation_5 (Activation)    (None, 500)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 1002      
_________________________________________________________________
activation_6 (Activation)    (None, 2)                 0         
Total params: 9,960,502
Trainable params: 9,959,502
Non-trainable params: 1,000
______________________________________________________________

In [ ]:
#preds = model.predict(DATA.tocsr())
#df_train_preds = pd.DataFrame(preds)
#df_train_preds.to_csv('vanechka_train_NOANAMN.csv')

# TEST DATA

In [ ]:
df = pd.read_csv('test_data.csv', sep=',')
df = df.fillna('NaN')
X_diag = df['Диагноз']
X_ana = df['Анамнез заболевания']
X_vnosm = df['Внешний осмотр']
X_age = df['Возраст'].values
X_sex = df['Пол'].values
df.head()

df_pr_1 = pd.read_csv('test_pred_1.csv', sep=',')
df_pr_2 = pd.read_csv('test_pred_2.csv', sep=',')
X_pr_1 = df_pr_1['proba'].values
X_pr_2 = df_pr_2['proba'].values

n_samples = len(X_diag.values)
print(n_samples)
corpus = np.hstack((X_diag.values, X_vnosm.values))
#corpus = np.hstack((X))
D = vectorizer.transform(corpus)

D_1 = D[:n_samples]
D_2 = D[n_samples:2*n_samples]
D_3 = D[-n_samples:]

#X_sex = np.asarray(preprocessing.scale(X_sex))
X_sex = X_sex[..., np.newaxis]
X_age = np.asarray(preprocessing.scale(X_age))
X_age = X_age[..., np.newaxis]
X_pr_1 = X_pr_1[..., np.newaxis]
X_pr_2 = X_pr_2[..., np.newaxis]

DATA_test = scipy.sparse.hstack((D_1, D_2, X_sex, X_age, X_pr_1, X_pr_2))

In [ ]:
print(DATA_test.shape)
final_preds = model.predict(DATA_test.tocsr())

In [ ]:
columns=['ID','proba']
print(final_preds[:,1])
a = np.vstack((np.array([i for i in range(final_preds.shape[0])], dtype=np.uint32), final_preds[:,1])).T
df = pd.DataFrame(a,columns=columns)
df.head()


In [ ]:
df.to_csv('vanechka_submission_NOANAMN_4.csv', index=False)

In [ ]:
!pip3 install h5py

In [ ]:
import h5py
model.save('vanechka_model_7.h5')

# Cheating

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('vanechka_submission_NOANAMN_3.csv', sep=',')
df.head()

p = df['proba'].values
print(np.mean(p), np.median(p))
plt.hist(p)

In [ ]:
for i in range(p.shape[0]):
    if (p[i]<0.8 and p[i]>0.25):
        p[i]-=0.25
plt.hist(p)
columns=['ID','proba']
a = np.vstack((np.array([i for i in range(p.shape[0])], dtype=np.uint32), p)).T
df = pd.DataFrame(a,columns=columns)
df.head()
df.to_csv('vanechka_submission_NOANAMN_3_cheat.csv', index=False)